<a href="https://colab.research.google.com/github/mahersalman/Introducation_To_Cloud_Computing/blob/main/Tutorials/Tutorial8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install firebase

In [ ]:
import nltk
import re
from collections import defaultdict
from nltk.chat.util import Chat, reflections
from firebase import firebase
from collections import Counter

# Initialize the connection to Firebase
firebase_url = 'https://cloud-tut6-2a99a-default-rtdb.firebaseio.com/'
FBconn = firebase.FirebaseApplication(firebase_url, None)

# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')

# Function to fetch data from Firebase based on file name
def fetch_data_by_file_name(file_name):
    result = FBconn.get('/JsonFiles', None)
    if result:
        for key, value in result.items():
            if value.get('file_name') == file_name:
                return value.get('data')  # Return the associated data
    return None

# Function to generate dynamic response - kkkkk
def generate_user_actions_response(user_input):
    match = re.match(r'what actions did (.*) perform\?', user_input)
    if match:
        user = match.group(1)
        # Filter the data to get only the actions performed by the specified user
        user_actions = [entry['Description'] for entry in data if entry['User'] == user]

        if user_actions:
            actions = ', '.join(user_actions)
            return f"{user} performed the following actions: {actions}"
        else:
            return f"No actions recorded for user '{user}'"
    return None

def generate_documents_accessed_response(user_input):
    match = re.match(r'can you list all the documents accessed by (.*)\?', user_input)
    if match:
        user = match.group(1)
        # Filter the data to get only the documents accessed by the specified user
        accessed_documents = set(entry['Document'] for entry in data if entry['User'] == user and 'Document' in entry)

        if accessed_documents:
            documents_list = '\n '.join(accessed_documents)
            return f"{user} accessed the following documents: {documents_list}"
        else:
            return f"No documents accessed by user '{user}'"
    return None


def generate_tab_access_response(user_input):
    match = re.match(r'which user accessed the (.*) tab the most\?', user_input)
    if match:
        tab = match.group(1)
        # Logic to find the user who accessed the specified tab the most
        if tab in tabs:
            # Assuming you have a data structure that maps users to tab access counts
            # For example: {'Tab1': {'User1': 10, 'User2': 5}}
            tab_users = {user: count for user, count in users.items() if tab in tabs}
            if tab_users:
                most_common_user = max(tab_users, key=tab_users.get)
                accesses = tab_users[most_common_user]
                return f"The user who accessed the {tab} tab the most is {most_common_user} with {accesses} accesses."
            else:
                return f"No user has accessed the {tab} tab."
        else:
            return f"No data available for the tab '{tab}'."
    return None

def generate_least_used_tab_response(user_input):
    match = re.match(r'which tab was used the least by (.*)\?', user_input)
    if match:
        user = match.group(1)
        # Filter the data to get only the entries relevant to the specified user
        user_tabs = defaultdict(int)
        for entry in data:
            if entry['User'] == user and entry['Tab'] != 'N/A':
                user_tabs[entry['Tab']] += 1

        if user_tabs:
            # Find the least used tab by the user
            least_used_tab = min(user_tabs, key=user_tabs.get)
            uses = user_tabs[least_used_tab]
            return f"The least used tab by {user} is '{least_used_tab}' with only {uses} uses."
        else:
            return f"{user} has not used any tabs or all actions are associated with 'N/A' tabs."
    return None



# Custom chatbot function - kkkkk
def custom_chatbot(user_input):
    if re.match(r'what actions did (.*) perform\?', user_input):
        return generate_user_actions_response(user_input)
    elif re.match(r'can you list all the documents accessed by (.*)\?', user_input):
        return generate_documents_accessed_response(user_input)
    elif re.match(r'which user accessed the (.*) tab the most\?', user_input):
        return generate_tab_access_response(user_input)
    elif re.match(r'which tab was used the least by (.*)\?', user_input):
        return generate_least_used_tab_response(user_input)
    else:
        return chatbot.respond(user_input)

# Fetch data from Firebase by file name
file_name = "OnshapeJson0.json"  # Specify the file name you're looking for
data = fetch_data_by_file_name(file_name)

if data:
    print("Data found:", data)
    #print(data.values())
else:
    print("No data found for the file name:", file_name)

# Process the data
users = Counter([item.get('User') for item in data])
documents = Counter([item.get('Document') for item in data if item.get('Document')])
tabs = Counter([item.get('Tab') for item in data if item.get('Tab')])
descriptions = Counter([item.get('Description') for item in data if item.get('Description')])

# Calculate some statistics
total_actions = len(data)
unique_users = len(users)
unique_documents = len(documents)
unique_tabs = len(tabs)

# Define patterns and responses
patterns = [
    (r'hi|hello|hey', ['Hello!', 'Hi there!', 'Welcome to the project management assistant.']),
    (r'how are you?', ['I\'m functioning well, thank you!', 'I\'m operational and ready to assist with your project management.']),
    (r'what is your name?', ['I\'m the Project Management Assistant.', 'You can call me the Assistant.']),
    (r'how do I use context in my project?', ['You can use context to manage state and share data across different parts of your application.']),
    (r'how do I define a type?', ['Defining types helps ensure that your variables are used correctly and can prevent errors in your application.']),
    (r'how can I use a keyboard shortcut?', ['Keyboard shortcuts can be defined to improve the efficiency of your application, allowing users to perform actions quickly.']),
    (r'how do I plan my project?', ['Planning your project involves setting clear goals, defining tasks, and creating a timeline to ensure that everything is completed on time.']),
    (r'how do I assemble my project components?', ['Assembling your project components involves integrating different parts of your application to work together seamlessly.']),
    (r'how can I sketch my ideas?', ['Sketching your ideas can help visualize your project before implementation, making it easier to plan and execute.']),
    (r'how do I draw diagrams for my project?', ['Drawing diagrams can help illustrate the structure and flow of your application, making it easier to understand and communicate.']),
    (r'what is a part in project management?', ['A part in project management refers to a component or segment of the overall project, which can be managed and tracked individually.']),
    (r'what is a studio in the context of development?', ['A studio in development refers to an integrated environment where various aspects of a project can be designed, developed, and tested.']),
    (r'exit|bye|goodbye', ['Thank you for using the Project Management Assistant. Goodbye!', 'Farewell! Don\'t hesitate to return if you need more assistance with your project.']),
    (r'how many total actions are recorded?', [f"There are {total_actions} total actions recorded in the database."]),
    (r'who is the most active user?', [f"The most active user is {users.most_common(1)[0][0]} with {users.most_common(1)[0][1]} actions."]),
    (r'how many unique users are there?', [f"There are {unique_users} unique users in the database."]),
    (r'What is the most frequently accessed document?', [f"The most frequently accessed document is '{documents.most_common(1)[0][0]}' with {documents.most_common(1)[0][1]} accesses."]),
    (r'how many different documents were accessed?', [f"There were {unique_documents} different documents accessed."]),
    (r'what is the most common tab used?', [f"The most commonly used tab is '{tabs.most_common(1)[0][0]}' with {tabs.most_common(1)[0][1]} uses."]),
    (r'what is the most frequent action description?', [f"The most frequent action description is '{descriptions.most_common(1)[0][0]}' occurring {descriptions.most_common(1)[0][1]} times."]),
    (r'what are the top 3 most active users?', [f"The top 3 most active users are: 1. {users.most_common(3)[0][0]} ({users.most_common(3)[0][1]} actions), 2. {users.most_common(3)[1][0]} ({users.most_common(3)[1][1]} actions), and 3. {users.most_common(3)[2][0]} ({users.most_common(3)[2][1]} actions)."]),
    (r'what is the least used tab?', [f"The least used tab is '{tabs.most_common()[-1][0]}' with only {tabs.most_common()[-1][1]} uses."]),
    (r'how many different types of actions \(descriptions\) are there?', [f"There are {len(descriptions)} different types of actions (unique descriptions) in the database."]),
    (r'what action did StudentA1 perform?', [f"StudentA1 performed actions like {descriptions.most_common(4)[2][0]}"]),

    (r'what actions did (.*) perform\?', 'DYNAMIC_RESPONSE'),
     (r'can you list all the documents accessed by (.*)\?', 'DYNAMIC_RESPONSE'),
    (r'which user accessed the (.*) tab the most\?', 'DYNAMIC_RESPONSE'),
    (r'which tab was used the least by (.*)\?', 'DYNAMIC_RESPONSE')
]

# Create a chatbot
chatbot = Chat(patterns, reflections)

# Start the conversation
print("Hello! I'm the Project's Chat Bot. How can I help you today?")
while True:
    user_input = input("You: ")

    if user_input.lower() in ['exit', 'bye', 'goodbye']:
        print("ChatBot: Thank you for using the Project Management Assistant. Goodbye!")
        break

    response = custom_chatbot(user_input)
    print("ChatBot:", response if response is not None else "This type of question is not supported.")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Data found: [{'Description': 'Close document', 'Document': 'Industry 4.0 Project', 'Tab': 'N/A', 'Time': '2023-01-13 14:12:54', 'User': 'StudentA1'}, {'Description': 'Tab Part Studio 2 Copy 1 of type PARTSTUDIO closed by StudentA1', 'Document': 'Industry 4.0 Project', 'Tab': 'Part Studio 2 Copy 1', 'Time': '2023-01-13 14:12:53', 'User': 'StudentA1'}, {'Description': 'Hide Right', 'Document': 'Industry 4.0 Project', 'Tab': 'Part Studio 2 Copy 1', 'Time': '2023-01-13 13:27:38', 'User': 'StudentA1'}, {'Description': 'Hide Front', 'Document': 'Industry 4.0 Project', 'Tab': 'Part Studio 2 Copy 1', 'Time': '2023-01-13 13:27:37', 'User': 'StudentA1'}, {'Description': 'Hide Top', 'Document': 'Industry 4.0 Project', 'Tab': 'Part Studio 2 Copy 1', 'Time': '2023-01-13 13:27:37', 'User': 'StudentA1'}, {'Description': 'Hide Origin', 'Document': 'Industry 4.0 Project', 'Tab': 'Part Studio 2 Copy 1', 'Time': '2023-01-13 13:27:36', 'User': 'StudentA1'}, {'Description': 'Tab Part Studio 2 Copy 1 of typ

KeyboardInterrupt: Interrupted by user